### 中醫醫案症狀詞抽取

- model: CRF
- 資料集: 舌象
- 類別標註: OS
- 詞位標注: 無
- 拆句: 以句點拆分
- 斷詞: 沒有做
- 詞性標註: 使用中科院的
- 引用辭典: 沒有
- 標點: 沒去

In [1]:
import json
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i >= ne_idx and i <= (ne_idx + ne_len - 1):
            return "S"
    
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    
    #原文
    case_content = cols[2]
    case_content = case_content.replace(" ","")
    case_content = case_content.replace('#',',')
    sentences = case_content.split("。")
    
    #摘要
    summary_str = cols[3].strip()
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "tongue" in keys:
        main = summary_json["tongue"]
        main_list = getTagListWithString(main)
        #print(main_list)
    
    t += len(main_list)

    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [16]:
#取2107筆
import random

rand_index = []
for x in range(3000):
    r = random.randint(0,48901)
    if len(rand_index)<2107 and r not in rand_index:
        rand_index.append(r)
        
docs2 = []
for idx in rand_index:
    docs2.append(docs[idx])
    
print(len(docs2))

2107


In [17]:
### 計算平均句長
total_wc = 0
for doc in docs2:
    total_wc += len(doc)
    
ave = total_wc / 2107
print(ave)

32.60702420503085


In [18]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器

data = []

for i, doc in enumerate(docs2):
    tokens = [t for t, label in doc]
    
    tagged = []
    for tk in tokens:
        tg = pynlpir.segment(tk)
        tagged.append(tg[0])
    
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [19]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    if postag is None:
        postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [20]:
# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
print(len(X_train))
print(len(X_test))

1685
422


In [22]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 6440
Seconds required: 0.084

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 3418.745258
Feature norm: 1.000000
Error norm: 4846.880786
Active features: 6420
Line search trials: 1
Line search step: 0.000011
Seconds required for this iteration: 0.021

***** Iteration #2 *****
Loss: 3166.712178
Feature norm: 1.053407
Error norm: 3788.622868
Active features: 6404
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #3 *****
Loss: 2624.443237
Feature norm: 1.243696
Error norm: 1440.483408
Active features: 4853
Line search trials: 1
Line search step: 1.000000
Seconds required for this iter

***** Iteration #55 *****
Loss: 179.698734
Feature norm: 18.661200
Error norm: 58.525568
Active features: 430
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.021

***** Iteration #56 *****
Loss: 179.044255
Feature norm: 18.814360
Error norm: 48.428960
Active features: 424
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.021

***** Iteration #57 *****
Loss: 178.589934
Feature norm: 19.012924
Error norm: 52.766952
Active features: 422
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.021

***** Iteration #58 *****
Loss: 178.273345
Feature norm: 19.085428
Error norm: 26.621491
Active features: 423
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.024

***** Iteration #59 *****
Loss: 178.020212
Feature norm: 19.134775
Error norm: 30.458622
Active features: 421
Line search trials: 2
Line search step: 0.500000
Seconds required for this ite

***** Iteration #98 *****
Loss: 173.714011
Feature norm: 20.323065
Error norm: 3.774532
Active features: 381
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #99 *****
Loss: 173.602419
Feature norm: 20.316054
Error norm: 8.902681
Active features: 372
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.012

***** Iteration #100 *****
Loss: 173.573935
Feature norm: 20.312881
Error norm: 23.040521
Active features: 366
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #101 *****
Loss: 173.521765
Feature norm: 20.312751
Error norm: 5.224301
Active features: 369
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.010

***** Iteration #102 *****
Loss: 173.485322
Feature norm: 20.307538
Error norm: 9.231060
Active features: 369
Line search trials: 1
Line search step: 1.000000
Seconds required for this iter

***** Iteration #142 *****
Loss: 172.367323
Feature norm: 20.700683
Error norm: 4.904917
Active features: 367
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.021

***** Iteration #143 *****
Loss: 172.362851
Feature norm: 20.715294
Error norm: 19.688068
Active features: 367
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #144 *****
Loss: 172.343558
Feature norm: 20.732936
Error norm: 15.864119
Active features: 365
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #145 *****
Loss: 172.312764
Feature norm: 20.743394
Error norm: 13.221942
Active features: 366
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #146 *****
Loss: 172.303329
Feature norm: 20.753614
Error norm: 12.160911
Active features: 366
Line search trials: 1
Line search step: 1.000000
Seconds required for this

***** Iteration #184 *****
Loss: 171.724769
Feature norm: 21.082411
Error norm: 5.149590
Active features: 364
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.021

***** Iteration #185 *****
Loss: 171.715017
Feature norm: 21.089114
Error norm: 9.652435
Active features: 364
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #186 *****
Loss: 171.713005
Feature norm: 21.094204
Error norm: 17.564951
Active features: 364
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.011

***** Iteration #187 *****
Loss: 171.696764
Feature norm: 21.105046
Error norm: 13.859337
Active features: 364
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.010

***** Iteration #188 *****
Loss: 171.684216
Feature norm: 21.109570
Error norm: 15.268456
Active features: 362
Line search trials: 1
Line search step: 1.000000
Seconds required for this 

In [23]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [24]:
# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print("F1 score for training set:\n")
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "S"]))

print("\n")

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print("F1 score for test set:\n")
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "S"]))

F1 score for training set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00     55059
          S       0.78      0.90      0.84       482

avg / total       1.00      1.00      1.00     55541



F1 score for test set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00     13036
          S       0.72      0.64      0.68       126

avg / total       0.99      0.99      0.99     13162

